# Import modules.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
import time
from stlab.devices.RS_SGS100A import RS_SGS100A

from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration_BMDevice import config, RR_1_IF,RO_lo, Q1_lo, R1_RS, Q1_RS, Q1_IF
#from Configuration_3D_RT_Cavity import config, RR_1_IF,RO_lo

# Define microwave sources

In [ ]:
RR = RS_SGS100A("TCPIP::169.254.184.193::INSTR", reset=True,verb=True) 
RR.EXTref()
RR.RFon()
RR.setCWpower(-15)
RR.setCWfrequency(RO_lo)
RR.write(':SOURce:IQ:IMPairment:LEAKage:I ' + R1_RS[0])
RR.write('SOURce:IQ:IMPairment:LEAKage:Q ' + R1_RS[1])
RR.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + R1_RS[2])
RR.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + R1_RS[3])
RR.IQon()
RR.write(':SOURce:IQ:IMPairment:STATe ON')

QDrive = RS_SGS100A("TCPIP::169.254.50.124::INSTR", reset=True,verb=True) 
QDrive.EXTref()
QDrive.RFon()
QDrive.setCWpower(-35)
QDrive.setCWfrequency(Q1_lo)
QDrive.write(':SOURce:IQ:IMPairment:LEAKage:I ' + Q1_RS[0])
QDrive.write('SOURce:IQ:IMPairment:LEAKage:Q ' + Q1_RS[1])
QDrive.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + Q1_RS[2])
QDrive.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + Q1_RS[3])
QDrive.IQon()
QDrive.write(':SOURce:IQ:IMPairment:STATe ON')

Off_Drive = RS_SGS100A("TCPIP::169.254.2.20::INSTR", reset=True,verb=True) 
Off_Drive.EXTref()
Off_Drive.RFon()
Off_Drive.setCWpower(-15)
Off_Drive.setCWfrequency(5.89e9)

# Define functions.

Defines a function that returns the correction matrix shown below.

Apply the ``g`` and ``phi`` which are basically the same kinds of scaling factors as in the RS I/Q impairment setting.

In [ ]:
def IQ_imbalance_corr(g, phi):
    c = np.cos(phi)
    s = np.sin(phi)
    N = 1 / ((1 - g ** 2) * (2 * c ** 2 - 1))
    return [float(N * x) for x in [(1 - g) * c, (1 + g) * s,
                                   (1 - g) * s, (1 + g) * c]]

# Configure QM unit.

In [ ]:
qmm = QuantumMachinesManager()
qm  = qmm.open_qm(config)

# Qubit control mixer correction.

Runs an infinite loop of pulses to a pre-defined element with a user-defined pulse amplitude, which simply scales the power coming out of the OPX

In [ ]:
f_q = 5.86544e9
qd_IF = int(f_q - Q1_lo)
 
f = qd_IF #(Q1_IF-4.0e6) #-20e6  
print(qd_IF/1e6)

QLO_Power = 0
OffResD_Power = -31.85
OffRes_D_f = 5.89e9

QDrive.setCWpower(QLO_Power)
Off_Drive.setCWpower(OffResD_Power)
Off_Drive.setCWfrequency(OffRes_D_f)

In [ ]:
with program() as mixer_cal_Qubit:
    update_frequency("Q1", f)
    with infinite_loop_():
        play("const"*amp(1), "Q1")
        #play("pi" * amp(1.2), "Q1", duration = 500)     #, duration=tau)

In [ ]:
job = qm.execute(mixer_cal_Qubit, duration_limit=0, data_limit=0)  #Start the above program.

In [ ]:
job.halt()  #Stop the program.

## 1. Mixer leakage correction.

You can run through the different values here for the ports of interest, which are defined through the settings of the "Q1_xy" element in the configuration file. 

The "I" of "Q1_xy" is defined in the configuration to be whichever port on the OPX that you wire it up to be here the number value for the calibration is what you want to enter as 'offset' in the controller for the corresponding port in the configuration file.

In [ ]:
qm.set_dc_offset_by_qe("Q1_xy", "I", -0.011)
qm.set_dc_offset_by_qe("Q1_xy", "Q", 0.0)

The above numbers should be saved as ``B_Q_corr_I`` and ``B_Q_corr_Q`` in the configuration file.

## 2. Mixer imbalance correction.

Returns the correction matrix that you should save as ``B_Q_corr_mat`` in configuration file.

In [ ]:
qm.set_mixer_correction('mixer_XY', int(Q_IF), int(Q_lo), IQ_imbalance_corr(0.0, -np.pi*0.0))

print(IQ_imbalance_corr(0.0, -np.pi*0.0))

# Readout input mixer calibration.

The principles are the same as in qubit control mixer calibration. No specific explanation given.

In [ ]:
with program() as mixer_cal_RR:
    
    with infinite_loop_():
        
        play("const"*amp(0.24), "RR_1") 

In [ ]:
job = qm.execute(mixer_cal_RR, duration_limit=0, data_limit=0)

In [ ]:
job.halt()

## 1. Mixer leakage correction.

In [ ]:
qm.set_dc_offset_by_qe("RR_1", "I",  0.00645)
qm.set_dc_offset_by_qe("RR_1", "Q", -0.00596)

## 2. Mixer imbalance correction.

In [ ]:
qm.set_mixer_correction('mixer_RO', int(RR_1_IF), int(RO_lo), IQ_imbalance_corr(0.017,np.pi*0.0415))

print(IQ_imbalance_corr(0.017,np.pi*0.0415))

# Readout output mixer calibration.

Observe the readout signal ... (to be updated).